In [1]:
# importing libraries
import re
import pandas as pd
import requests
from urllib.request import urlopen as ureq
from requests import get
from bs4 import BeautifulSoup as BS
import time
import numpy as np
from urllib.parse import urlparse
from tqdm import tqdm_notebook as tqdm

In [2]:
#creating dataframe
df = pd.DataFrame(columns=['ID', 'Country'])

In [3]:
r = requests.get('http://www.espncricinfo.com/australia/content/player/index.html?country=2')
soup = BS(r.content,"html5lib")
rows = soup.find("ul", attrs={'id':'ciPlayerhomecntry'}).find_all("li")
for row in rows:
        if row.text!='Other nations':
            c_id = re.search('\d+',str(row.find("a", href=re.compile("^index.html")))).group(0)
            country = row.text.strip()
            df = df.append(pd.Series([str(c_id), country], index=df.columns ), ignore_index=True)
# df

In [4]:
r = requests.get('http://www.espncricinfo.com/australia/content/player/index.html?country=2')
soup = BS(r.content,"html5lib")
rows = soup.find_all("option")
for row in rows:
        if row.text!='Select team' and row.text!='':
            c_id = row['value']
            country = row.text.strip()
            df = df.append(pd.Series([str(c_id), country], index=df.columns ), ignore_index=True)
# df

In [5]:
df_new = pd.DataFrame(columns=['Tests', 'ODIs', 'T20'])

In [6]:
for i in tqdm(range(0,len(df['ID'])),desc="Progress",total=len(df['ID'])):
    r = requests.get('http://www.espncricinfo.com/australia/content/player/index.html?country='+df['ID'][i])
    soup = BS(r.content,"html5lib") 
    if soup.find("p", attrs={'class':'ciPhotoWidgetLink'}).text=='Recent Players:':
        rows = soup.find("ul", attrs={'id':'bwsPlytab'}).find_all("li")
        for row in rows:
            if row.text=='Tests':
                df_new.loc[i,'Tests'] = 1  
            if row.text=='ODIs':
                df_new.loc[i,'ODIs'] = 1
            if row.text=='T20':
                df_new.loc[i,'T20'] = 1 
    else:
        df_new.loc[i,'Tests'] = 0  
        df_new.loc[i,'ODIs'] = 0
        df_new.loc[i,'T20'] = 0 
#             country = row.text.strip()
#             df = df.append(pd.Series([str(c_id), country], index=df.columns ), ignore_index=True)
# df_new

In [7]:
df['Tests']= df_new['Tests'].fillna(0).astype(int)
df['ODIs']= df_new['ODIs'].fillna(0).astype(int)
df['T20']= df_new['T20'].fillna(0).astype(int)

In [8]:
player_list = pd.DataFrame(columns=['p_id', 'Name', 'Country', 'c_id'])

In [9]:
for i in tqdm(range(0,len(df['ID'])),desc="Progress",total=len(df['ID'])):
    if df['Tests'][i] == 1:
        r = requests.get('http://www.espncricinfo.com/india/content/player/caps.html?country='+df['ID'][i]+';class=1')
        soup = BS(r.content,"html5lib") 
        rows = soup.find_all("a", href=re.compile("^/india/content/player"))
        for row in rows[1:]:
            p_id = re.search('\d+',str(row)).group(0)
            player_url = 'http://www.espncricinfo.com'+row['href']
            r_player = requests.get(player_url)
            soup_player = BS(r_player.content,"html5lib")
            try:
              table_player = soup_player.find("div", attrs={'style':'float:left; width:310px; color:#666666; font-size:11px;'}).find('p', attrs={'class':'ciPlayerinformationtxt'}).find('span')
              name = table_player.text.strip()
            except:
              name = row.text.strip()            
            player_list = player_list.append(pd.Series([p_id, name, df['Country'][i], df['ID'][i]],  index=player_list.columns ), ignore_index=True )
            
    if df['ODIs'][i] == 1:
        r = requests.get('http://www.espncricinfo.com/india/content/player/caps.html?country='+df['ID'][i]+';class=2')
        soup = BS(r.content,"html5lib") 
        rows = soup.find_all("a", href=re.compile("^/india/content/player"))
        for row in rows[1:]:
            p_id = re.search('\d+',str(row)).group(0)
            player_url = 'http://www.espncricinfo.com'+row['href']
            r_player = requests.get(player_url)
            soup_player = BS(r_player.content,"html5lib")
            try:
              table_player = soup_player.find("div", attrs={'style':'float:left; width:310px; color:#666666; font-size:11px;'}).find('p', attrs={'class':'ciPlayerinformationtxt'}).find('span')
              name = table_player.text.strip()
            except:
              name = row.text.strip()
            player_list = player_list.append(pd.Series([p_id, name, df['Country'][i], df['ID'][i]],  index=player_list.columns ), ignore_index=True )

    if df['T20'][i] == 1:
        r = requests.get('http://www.espncricinfo.com/india/content/player/caps.html?country='+df['ID'][i]+';class=3')
        soup = BS(r.content,"html5lib") 
        rows = soup.find_all("a", href=re.compile("^/india/content/player"))
        for row in rows[1:]:
            p_id = re.search('\d+',str(row)).group(0)
            player_url = 'http://www.espncricinfo.com'+row['href']
            r_player = requests.get(player_url)
            soup_player = BS(r_player.content,"html5lib")
            try:
              table_player = soup_player.find("div", attrs={'style':'float:left; width:310px; color:#666666; font-size:11px;'}).find('p', attrs={'class':'ciPlayerinformationtxt'}).find('span')
              name = table_player.text.strip()
            except:
              name = row.text.strip()
            player_list = player_list.append(pd.Series([p_id, name, df['Country'][i], df['ID'][i]],  index=player_list.columns ), ignore_index=True )

In [10]:
player_list = player_list.drop_duplicates()
player_list.to_csv('list_of_players.csv',index=False)

In [11]:
player_list = pd.read_csv('list_of_players.csv',header=0,converters={'ID': lambda x: str(x)})

In [12]:
year_columns = []
for i in range(1877,2020):
    year_columns.append(str(i))

In [13]:
data_4s = pd.concat([player_list,pd.DataFrame(columns=year_columns)], axis=1)
data_4s = data_4s.fillna(0)
data_4s['p_id'] = data_4s['p_id'].astype(int)

data_6s = pd.concat([player_list,pd.DataFrame(columns=year_columns)], axis=1)
data_6s = data_6s.fillna(0)
data_6s['p_id'] = data_6s['p_id'].astype(int)

In [14]:
for i in tqdm(range(0,len(data_4s['p_id'])),desc="Progress",total=len(data_4s['p_id'])):
    flag = True
    temp_data = pd.DataFrame(columns=['4s', '6s','Year'])
    r = requests.get('http://stats.espncricinfo.com/ci/engine/player/'+str(int(data_4s['p_id'][i]))+'.html?class=1;template=results;type=batting;view=innings')
    soup = BS(r.content,"html5lib") 
    error = soup.find("h1", attrs={'class':'Error404__Title'})
    if str(error) == 'None':
        # rows = soup.find("table", attrs={'class':'TableLined'}).find("tbody").find_all("tr")
        for caption in soup.find_all('caption'):
            if caption.get_text() == 'Innings by innings list':
                rows = caption.find_parent('table').find("tbody").find_all("tr")
                break
        for row in rows:
            if row.text.strip() == 'No records available to match this query':
                flag = False
                pass
            else:
                for a in range(0,1):
                    fours = row.find("td").find_next('td').find_next('td').find_next('td').text.strip()
                    sixes = row.find("td").find_next('td').find_next('td').find_next('td').find_next('td').text.strip()
                    year = row.find("td").find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').text.strip()
                    year = year[-4:]
                    if fours == '-':
                        fours = int(0)
                    if sixes == '-':
                        sixes = int(0)
                    temp_data = temp_data.append(pd.Series([fours, sixes, year],  index=temp_data.columns ), ignore_index=True )
        if flag:
            temp_data['4s'] = temp_data['4s'].astype(int)
            temp_data['6s'] = temp_data['6s'].astype(int)
            new_temp = temp_data.groupby('Year')['4s','6s'].agg('sum')
            new_temp.reset_index(level=0, inplace=True)
            for j in range(0,len(new_temp['4s'])):
                data_4s.loc[i,'%s' % new_temp['Year'][j]] = new_temp['4s'][j]
                data_6s.loc[i,'%s' % new_temp['Year'][j]] = new_temp['6s'][j]

    flag2 = True
    temp_data2 = pd.DataFrame(columns=['4s', '6s','Year'])
    r2 = requests.get('http://stats.espncricinfo.com/ci/engine/player/'+str(int(data_4s['p_id'][i]))+'.html?class=2;template=results;type=batting;view=innings')
    soup2 = BS(r2.content,"html5lib") 
    error2 = soup2.find("h1", attrs={'class':'Error404__Title'})
    if str(error2) == 'None':
        # rows = soup.find("table", attrs={'class':'TableLined'}).find("tbody").find_all("tr")
        for caption in soup2.find_all('caption'):
            if caption.get_text() == 'Innings by innings list':
                rows2 = caption.find_parent('table').find("tbody").find_all("tr")
                break
        for row in rows2:
            if row.text.strip() == 'No records available to match this query':
                flag2 = False
                pass
            else:
                for a in range(0,1):
                    fours = row.find("td").find_next('td').find_next('td').find_next('td').text.strip()
                    sixes = row.find("td").find_next('td').find_next('td').find_next('td').find_next('td').text.strip()
                    year = row.find("td").find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').text.strip()
                    year = year[-4:]
                    if fours == '-':
                        fours = int(0)
                    if sixes == '-':
                        sixes = int(0)
                    temp_data2 = temp_data2.append(pd.Series([fours, sixes, year],  index=temp_data2.columns ), ignore_index=True )
        if flag2:
            temp_data2['4s'] = temp_data2['4s'].astype(int)
            temp_data2['6s'] = temp_data2['6s'].astype(int)
            new_temp2 = temp_data2.groupby('Year')['4s','6s'].agg('sum')
            new_temp2.reset_index(level=0, inplace=True)
            for j in range(0,len(new_temp2['4s'])):
                data_4s.loc[i,'%s' % new_temp2['Year'][j]] += new_temp2['4s'][j]
                data_6s.loc[i,'%s' % new_temp2['Year'][j]] += new_temp2['6s'][j]

    flag3 = True
    temp_data3 = pd.DataFrame(columns=['4s', '6s','Year'])
    r3 = requests.get('http://stats.espncricinfo.com/ci/engine/player/'+str(int(data_4s['p_id'][i]))+'.html?class=3;template=results;type=batting;view=innings')
    soup3 = BS(r3.content,"html5lib") 
    error3 = soup3.find("h1", attrs={'class':'Error404__Title'})
    if str(error3) == 'None':
        # rows = soup.find("table", attrs={'class':'TableLined'}).find("tbody").find_all("tr")
        for caption in soup3.find_all('caption'):
            if caption.get_text() == 'Innings by innings list':
                rows3 = caption.find_parent('table').find("tbody").find_all("tr")
                break
        for row in rows3:
            if row.text.strip() == 'No records available to match this query':
                flag3 = False
                pass
            else:
                for a in range(0,1):
                    fours = row.find("td").find_next('td').find_next('td').find_next('td').text.strip()
                    sixes = row.find("td").find_next('td').find_next('td').find_next('td').find_next('td').text.strip()
                    year = row.find("td").find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').find_next('td').text.strip()
                    year = year[-4:]
                    if fours == '-':
                        fours = int(0)
                    if sixes == '-':
                        sixes = int(0)
                    temp_data3 = temp_data3.append(pd.Series([fours, sixes, year],  index=temp_data3.columns ), ignore_index=True )
        if flag3:
            temp_data3['4s'] = temp_data3['4s'].astype(int)
            temp_data3['6s'] = temp_data3['6s'].astype(int)
            new_temp3 = temp_data3.groupby('Year')['4s','6s'].agg('sum')
            new_temp3.reset_index(level=0, inplace=True)
            for j in range(0,len(new_temp3['4s'])):
                data_4s.loc[i,'%s' % new_temp3['Year'][j]] += new_temp3['4s'][j]
                data_6s.loc[i,'%s' % new_temp3['Year'][j]] += new_temp3['6s'][j]

In [15]:
#funtion to get the total no. of boundaries scored between 1877 and some Year by a Player
def get_boundaries_per_player(player, year):
    data_6 = pd.DataFrame()
    data_4 = pd.DataFrame()

    data_6 = data_6s.loc[data_6s['Name'] == player].values
    data_4 = data_4s.loc[data_4s['Name'] == player].values

    mapping = dict(zip(range(5,148),range(1877,2020)))
    val = list(mapping.keys())[list(mapping.values()).index(year)]
    tot_4s = np.sum(data_4[:,4:val].astype(int))
    tot_6s = np.sum(data_6[:,4:val].astype(int))
    return tot_4s, tot_6s
#     print("Total boundaries scored by {0} from 1877 to {1}: 4s - {2}, 6s - {3}".format(player, year, tot_4s, tot_6s))

In [16]:
for i in range(len(data_4s['Name'])):
    tot_4, tot_6 = get_boundaries_per_player(data_4s['Name'][i],2019) #please specify name of the player just as it is in the Dataframe/CSV for proper output
    data_4s.loc[i,'Total'] = tot_4
    data_6s.loc[i,'Total'] = tot_6

In [17]:
data_4s['Total'] = data_4s['Total'].astype(int)
data_6s['Total'] = data_6s['Total'].astype(int)

In [18]:
data_4s

,p_id,Name,Country,c_id,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,...,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total
0,4091,Charles Bannerman,Australia,2,19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19
1,4153,John McCarthy Blackham,Australia,2,0,0,0,2,2,9,14,8,1,0,1,1,0,0,0,2,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,48
2,4625,Bransby Beauchamp Cooper,Australia,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5364,Thomas William Garrett,Australia,2,0,0,0,0,0,2,0,0,8,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12
4,5432,David William Gregory,Australia,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5433,Edward James Gregory,Australia,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,5670,John Robart Hodges,Australia,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
7,5705,Thomas Patrick Horan,Australia,2,2,0,0,0,10,2,3,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27
8,6141,Thomas Kingston Kendall,Australia,2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3
9,6605,William Evans Midwinter,Australia,2,1,0,0,0,5,2,1,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28


In [19]:
data_6s

,p_id,Name,Country,c_id,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,...,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total
0,4091,Charles Bannerman,Australia,2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,4153,John McCarthy Blackham,Australia,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,4625,Bransby Beauchamp Cooper,Australia,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,5364,Thomas William Garrett,Australia,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,5432,David William Gregory,Australia,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,5433,Edward James Gregory,Australia,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,5670,John Robart Hodges,Australia,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,5705,Thomas Patrick Horan,Australia,2,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,6141,Thomas Kingston Kendall,Australia,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,6605,William Evans Midwinter,Australia,2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2


In [20]:
fours_by_country = data_4s.groupby('Country')[year_columns].agg('sum')
fours_by_country = fours_by_country.reset_index(level=0)
fours_by_country['Total'] = 0

sixes_by_country = data_6s.groupby('Country')[year_columns].agg('sum')
sixes_by_country = sixes_by_country.reset_index(level=0)
sixes_by_country['Total'] = 0

In [21]:
#funtion to get the total no. of boundaries scored in a year by all players
def get_boundaries_per_year(year):
    data_6 = pd.DataFrame()
    data_4 = pd.DataFrame()

    data_6 = data_6s[''+str(year)].values
    data_4 = data_4s[''+str(year)].values


    tot_4s = np.sum(data_4.astype(int))
    tot_6s = np.sum(data_6.astype(int))

    print("Total boundaries scored in {0}: 4s - {1}, 6s - {2}".format(year, tot_4s, tot_6s))

In [22]:
get_boundaries_per_year(1877)

Total boundaries scored in 1877: 4s - 64, 6s - 2


In [23]:
for i in range(0,len(fours_by_country['1877'])):
    fours_by_country['Total'][i] = fours_by_country.iloc[i,1:].astype(int).sum()
    sixes_by_country['Total'][i] = sixes_by_country.iloc[i,1:].astype(int).sum()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [24]:
sixes_by_country

,Country,1877,1878,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1899,1900,1901,1902,1903,1904,1905,1906,1907,1908,1909,1910,1911,1912,1913,1914,1915,...,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,Total
0,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8,69,13,47,65,85,147,146,132,148,115,975
1,Australia,1,0,0,0,1,7,5,0,7,0,2,0,0,0,0,5,0,4,2,0,0,3,0,0,0,7,0,5,2,0,0,3,5,1,3,5,0,0,0,...,21,30,41,21,42,44,42,11,38,49,29,29,40,34,33,36,49,64,106,72,115,123,160,131,176,162,224,96,203,225,151,214,238,223,171,248,148,191,113,4630
2,Bahrain,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,30,30
3,Bangladesh,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,10,3,8,15,22,9,24,53,39,81,101,55,79,97,48,81,83,123,100,106,101,183,65,1487
4,Belgium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,15
5,Belize,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,23
6,Bermuda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,28,66,17,5,0,0,0,0,0,0,0,0,0,0,116
7,Botswana,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5
8,Canada,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,19,0,0,13,47,47,21,32,28,15,11,3,0,0,0,0,0,242
9,Costa Rica,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,7


In [25]:
data_4s.to_csv('all_4s.csv',index=False)
data_6s.to_csv('all_6s.csv',index=False)

In [26]:
fours_by_country.to_csv('fours_by_country.csv',index=False)
sixes_by_country.to_csv('sixes_by_country.csv',index=False)